In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, ModelSettings, trace, function_tool

import pandas as pd
import PyPDF2
import asyncio

from IPython.display import Markdown, display

load_dotenv(override=True)

True

In [2]:
INSTRUCTIONS = """
You are a data analyst reviewing an Excel file against PDF specifications. Provide a comprehensive analysis with precise issue locations and actionable recommendations.

## Analysis Scope

1. **Structure Compliance**: Verify sheets, columns, headers, data types, and field requirements match PDF specs
2. **Format Validation**: Check dates, numbers, text, codes, and boolean fields for correct formatting
3. **Data Consistency**: Validate temporal logic, numerical relationships, business rules, and data integrity
4. **Quality Checks**: Identify missing data, duplicates, outliers, and formatting issues

## For Each Issue Report

- **Location**: Sheet name, cell reference (e.g., "C15"), column name, row number
- **Type**: Format violation | Consistency error | Missing data | Invalid value | Structural problem
- **Severity**: Critical (blocks processing) | Major (significant deviation) | Minor (formatting)
- **Details**: Current value vs. expected value with clear explanation
- **Fix**: Specific Excel-based solution (no external code)

## Output Structure

EXECUTIVE SUMMARY
- Total issues by severity
- Key problem areas

DETAILED FINDINGS
Sheet: [Name]
Issue #1: [Type] - [Severity]
• Location: Cell B7, Column "Date"
• Found: "2024/13/45"
• Expected: Valid date (DD/MM/YYYY)
• Description: Invalid date format
• Fix: Correct date entry


## Key Instructions

- Compare against PDF examples/templates
- Prioritize data integrity issues
- Note where file exceeds requirements
- Flag unclear PDF sections
- Skip 'data dictionary' sheet
- Focus on Excel-native solutions only
- Make sure that any recommendations you make, can be done in Excel. Do not suggest the use of Python or any other code language to fix the identified issues.
- Ignore 'data dictionary' sheet. No analysis should be done on this sheet!
"""

In [3]:
@function_tool
def read_pdf_specs(pdf_path: str) -> str:
    """Read PDF specification file and extract text content.
    
    Args:
        pdf_path: Path to the PDF specification file
    """
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        content = []
        for page in pdf_reader.pages:
            content.append(page.extract_text())
    return "\n".join(content)


def detect_date_format_inconsistency(series):
        """Ultra-simple date format consistency check."""
        if series.dtype != 'object':
            return False, set(), []
        
        sample = series.dropna().head(10)
        warnings = []
        
        # Try parsing with different formats - pandas will detect inconsistencies
        try:
            parsed1 = pd.to_datetime(sample, format='%d/%m/%Y', errors='coerce')
            parsed2 = pd.to_datetime(sample, format='%m/%d/%Y', errors='coerce')
            
            # Check if same strings parse to different dates
            both_valid = pd.notna(parsed1) & pd.notna(parsed2)
            different_results = parsed1 != parsed2
            
            if (both_valid & different_results).any():
                warnings.append("Inconsistent date formats detected (DD/MM vs MM/DD)")
                return True, {'MIXED_FORMATS'}, warnings
                
        except:
            pass
        
        return False, set(), warnings

@function_tool  
def read_excel_data(excel_path: str) -> str:
    """Read Excel file and return structure and sample data.
    
    Args:
        excel_path: Path to the Excel file to analyze
    """
    excel_data = pd.read_excel(excel_path, sheet_name=None)
    
    analysis = []
    for sheet_name, df in excel_data.items():
            analysis.append(f"=== SHEET: {sheet_name} ===")
            analysis.append(f"Shape: {df.shape[0]} rows, {df.shape[1]} columns")
            analysis.append(f"Columns: {list(df.columns)}")
            analysis.append(f"Data types:\n{df.dtypes}")
            
            # Enhanced data quality metrics
            analysis.append(f"\n--- DATA QUALITY SUMMARY ---")
            analysis.append(f"Missing values per column:\n{df.isnull().sum()}")
            analysis.append(f"Missing values percentage:\n{(df.isnull().sum() / len(df) * 100).round(2)}%")
            analysis.append(f"Duplicate rows: {df.duplicated().sum()}")
            
            # More comprehensive sampling
            analysis.append(f"\n--- DATA SAMPLES ---")
            analysis.append(f"First 3 rows:\n{df.head(3)}")
            if len(df) > 6:
                analysis.append(f"Last 3 rows:\n{df.tail(3)}")
            if len(df) > 10:
                analysis.append(f"Random sample (3 rows):\n{df.sample(min(3, len(df)-6), random_state=42)}")
            
            # Column-specific analysis
            analysis.append(f"\n--- COLUMN ANALYSIS ---")
            for col in df.columns:
                col_info = []
                col_info.append(f"Column '{col}':")
                col_info.append(f"  - Type: {df[col].dtype}")
                col_info.append(f"  - Non-null count: {df[col].count()}")
                col_info.append(f"  - Unique values: {df[col].nunique()}")
                
                if df[col].dtype in ['object', 'string']:
                    # Text analysis
                    unique_vals = df[col].dropna().unique()
                    if len(unique_vals) <= 10:
                        col_info.append(f"  - Unique values: {list(unique_vals)}")
                    else:
                        col_info.append(f"  - Sample unique values: {list(unique_vals[:10])}")
                    
                    # Check for potential formatting issues
                    has_mixed_case = len(set(str(v).lower() if pd.notna(v) else v for v in df[col].unique())) != df[col].nunique()
                    if has_mixed_case:
                        col_info.append(f"  - WARNING: Mixed case values detected")
                    
                    # Check for leading/trailing spaces
                    if df[col].astype(str).str.strip().nunique() != df[col].nunique():
                        col_info.append(f"  - WARNING: Leading/trailing spaces detected")
                    
                    # Check for date format inconsistency
                    inconsistent, formats, warnings = detect_date_format_inconsistency(df[col])
                    if inconsistent or warnings:
                        col_info.append(f"  - DATE FORMAT ISSUES:")
                        for warning in warnings:
                            col_info.append(f"    - WARNING: {warning}")
                
                elif df[col].dtype in ['int64', 'float64', 'int32', 'float32']:
                    # Numeric analysis
                    col_info.append(f"  - Min: {df[col].min()}")
                    col_info.append(f"  - Max: {df[col].max()}")
                    col_info.append(f"  - Mean: {df[col].mean():.2f}")
                    col_info.append(f"  - Median: {df[col].median()}")
                    
                    # Check for outliers (simple IQR method)
                    Q1 = df[col].quantile(0.25)
                    Q3 = df[col].quantile(0.75)
                    IQR = Q3 - Q1
                    outliers = df[(df[col] < (Q1 - 1.5 * IQR)) | (df[col] > (Q3 + 1.5 * IQR))][col]
                    if len(outliers) > 0:
                        col_info.append(f"  - Potential outliers: {len(outliers)} values")
                        col_info.append(f"  - Outlier range: {outliers.min()} to {outliers.max()}")
                
                elif df[col].dtype in ['datetime64[ns]', 'datetime64']:
                    # Date analysis
                    col_info.append(f"  - Date range: {df[col].min()} to {df[col].max()}")
                
                analysis.append("\n".join(col_info))
            
            analysis.append("")
    
    return "\n".join(analysis)

In [4]:
data_qc_agent = Agent(
    name="data_qc_agent",
    instructions=INSTRUCTIONS,
    tools=[read_pdf_specs, read_excel_data], 
    model="gpt-4o-mini",
    model_settings=ModelSettings(
        tool_choice="required"
    )
)

In [5]:
data_qc_agent 

Agent(name='data_qc_agent', instructions='\nYou are a data analyst reviewing an Excel file against PDF specifications. Provide a comprehensive analysis with precise issue locations and actionable recommendations.\n\n## Analysis Scope\n\n1. **Structure Compliance**: Verify sheets, columns, headers, data types, and field requirements match PDF specs\n2. **Format Validation**: Check dates, numbers, text, codes, and boolean fields for correct formatting\n3. **Data Consistency**: Validate temporal logic, numerical relationships, business rules, and data integrity\n4. **Quality Checks**: Identify missing data, duplicates, outliers, and formatting issues\n\n## For Each Issue Report\n\n- **Location**: Sheet name, cell reference (e.g., "C15"), column name, row number\n- **Type**: Format violation | Consistency error | Missing data | Invalid value | Structural problem\n- **Severity**: Critical (blocks processing) | Major (significant deviation) | Minor (formatting)\n- **Details**: Current value 

In [6]:
async def test_data_qc():
    result = await Runner.run(
        data_qc_agent, 
        """
        Execute this data quality analysis workflow:
        
        1. FIRST: Call read_pdf_specs with file path 'spec.pdf' to get the specifications
        2. SECOND: Call read_excel_data with file path 'data.xlsx' to analyze the Excel file
        3. THIRD: Compare findings and provide detailed quality assessment
        
        Start by using the read_pdf_specs tool now.
        """,
        max_turns=5
    )
    
    display(Markdown(result.final_output))

In [7]:
# Run the test
with trace("Data QC Agent Test 1"):
    await test_data_qc()

## EXECUTIVE SUMMARY

### Total Issues by Severity
- **Critical**: 5
- **Major**: 7
- **Minor**: 6

### Key Problem Areas
- **Customer Data**: Invalid entries, formatting issues, and missing required fields.
- **Orders Sheet**: Date format issues and potential integrity problems with TotalAmount.
- **Products Sheet**: Invalid values in StockQuantity and Price, and date formatting inconsistencies.

---

## DETAILED FINDINGS

### Sheet: Customer Data

**Issue #1: Structural Problem - Major**
- **Location**: Cell B3, Column "CustomerID"
- **Found**: NaN
- **Expected**: Required field with format "CUST-XXXXX"
- **Description**: Missing CustomerID is critical for data integrity.
- **Fix**: Populate with a valid CustomerID in the format "CUST-XXXXX".

**Issue #2: Format Violation - Major**
- **Location**: Cell B4, Column "DateJoined"
- **Found**: "2024/13/45"
- **Expected**: Valid date (DD/MM/YYYY)
- **Description**: Invalid date format.
- **Fix**: Correct to a valid date (e.g., "13/12/2024").

**Issue #3: Missing Data - Major**
- **Location**: Cell D5, Column "Phone"
- **Found**: Empty
- **Expected**: Phone (Optional but preferred)
- **Description**: Optional field missing.
- **Fix**: Add a valid phone number in the format "(XXX) XXX-XXXX".

**Issue #4: Consistency Error - Major**
- **Location**: Cell G3, Column "Status"
- **Found**: "Invalid"
- **Expected**: Values "Active", "Inactive", "Pending"
- **Description**: Invalid status value.
- **Fix**: Change value to one of the allowed statuses.

**Issue #5: Duplicate Rows - Major**
- **Location**: Row 5
- **Description**: Duplicate entry detected for CustomerID "CUST-12345".
- **Fix**: Remove the duplicate entry.

---

### Sheet: Orders

**Issue #6: Structural Problem - Major**
- **Location**: Cell A3, Column "OrderID"
- **Found**: NaN
- **Expected**: Required field
- **Description**: Missing OrderID critical for data processing.
- **Fix**: Populate with a valid OrderID in the format "ORD-XXXXXXXX".

**Issue #7: Format Violation - Major**
- **Location**: Cell C5, Column "OrderDate"
- **Found**: "32/13/2023"
- **Expected**: Valid date (DD/MM/YYYY)
- **Description**: Invalid date.
- **Fix**: Correct to a valid date (e.g., "31/12/2023").

**Issue #8: Invalid Value - Critical**
- **Location**: Cell D5, Column "TotalAmount"
- **Found**: 450000
- **Expected**: Must be less than 100,000
- **Description**: TotalAmount exceeds maximum allowed value.
- **Fix**: Adjust value to fall within range (e.g., 450).

---

### Sheet: Products

**Issue #9: Missing Data - Major**
- **Location**: Cell A3, Column "ProductCode"
- **Found**: NaN
- **Expected**: Required field
- **Description**: Missing ProductCode.
- **Fix**: Provide a valid product code in the format "PROD-XXX-YYY".

**Issue #10: Invalid Value - Major**
- **Location**: Cell E3, Column "StockQuantity"
- **Found**: -10
- **Expected**: Non-negative integer.
- **Description**: Invalid negative quantity.
- **Fix**: Set to a non-negative integer.

**Issue #11: Format Violation - Minor**
- **Location**: Cell F4, Column "LastUpdated"
- **Found**: "30/02/2024"
- **Expected**: Valid date (DD/MM/YYYY)
- **Description**: Invalid date entry.
- **Fix**: Change to a valid date (e.g., "01/03/2024").

**Issue #12: Date Format Issues - Minor**
- **Location**: Multiple Cells, all date columns
- **Description**: Inconsistent date formats across sheets.
- **Fix**: Standardize all date entries to DD/MM/YYYY across all relevant columns.

---

### General Recommendations

1. **Standardize Data Entry**: Use Excel's data validation to ensure entries are consistent with specifications.
2. **Remove Duplicates Regularly**: Implement a regular check for duplicates in key columns (like CustomerID, OrderID).
3. **Training**: Educate data entry personnel on requirements, especially regarding mandatory fields and format specifications.

In [8]:
# Run the test
with trace("Data QC Agent Test 2"):
    await test_data_qc()

### EXECUTIVE SUMMARY

- **Total Issues by Severity:**
  - Critical: 5
  - Major: 8
  - Minor: 7

- **Key Problem Areas:**
  1. **Customer Data**: Invalid data formats, missing values, and duplicate entries.
  2. **Orders**: Invalid values in OrderID and TotalAmount, and date inconsistencies.
  3. **Products**: Missing ProductCode, inconsistent pricing formats, and date inconsistencies.

---

### DETAILED FINDINGS

---

#### **Sheet: Customer Data**
- **Issue #1**: Missing Data - **Major**
   - **Location**: Cell A3 (Row 3)
   - **Found**: Empty
   - **Expected**: Valid CustomerID
   - **Description**: Missing CustomerID for a valid entry.
   - **Fix**: Ensure all customer records have a unique CustomerID.

- **Issue #2**: Format Violation - **Critical**
   - **Location**: Cell F2 (Row 2), Column "DateJoined"
   - **Found**: "2024/13/45"
   - **Expected**: Valid date format (DD/MM/YYYY)
   - **Description**: Invalid date format detected.
   - **Fix**: Correct date entry to follow DD/MM/YYYY.

- **Issue #3**: Format Violation - **Major**
   - **Location**: Cell F2 (Row 2), Column "DateJoined"
   - **Found**: "2024/13/45"
   - **Expected**: valid date format (must be between 01/01/2020 and 31/12/2025).
   - **Description**: Date is invalid (month and day exceed valid range).
   - **Fix**: Adjust date to a valid DD/MM/YYYY format.

- **Issue #4**: Duplicate Entry - **Major**
   - **Location**: Row 1
   - **Found**: CustomerID "CUST-12345"
   - **Expected**: Unique CustomerID
   - **Description**: Duplicate CustomerID entries detected.
   - **Fix**: Remove duplicates or amend to ensure unique CustomerIDs.

- **Issue #5**: Format Violation - **Minor**
   - **Location**: Cell E2 (Row 2), Column "Phone"
   - **Found**: "555-234-5678"
   - **Expected**: Format "(XXX) XXX-XXXX"
   - **Description**: Phone number does not match expected format.
   - **Fix**: Reformat the phone number to match "(XXX) XXX-XXXX".

---

#### **Sheet: Orders**
- **Issue #1**: Missing Data - **Major**
   - **Location**: Cell A3 (Row 3)
   - **Found**: Empty
   - **Expected**: Valid OrderID
   - **Description**: Missing OrderID for an entry.
   - **Fix**: Populate OrderID for all orders.

- **Issue #2**: Format Violation - **Critical**
   - **Location**: Cell C4 (Row 4), Column "TotalAmount"
   - **Found**: 450000
   - **Expected**: Decimal less than 99999.99
   - **Description**: TotalAmount exceeds maximum value.
   - **Fix**: Correct the amount to be less than 99999.99.

- **Issue #3**: Format Violation - **Major**
   - **Location**: Cell B3 (Row 3), Column "OrderDate"
   - **Found**: "32/13/2023"
   - **Expected**: Valid date format (DD/MM/YYYY)
   - **Description**: Invalid date format.
   - **Fix**: Correct the date to a valid DD/MM/YYYY format.

- **Issue #4**: Format Violation - **Minor**
   - **Location**: Cell E2 (Row 2), Column "PaymentMethod"
   - **Found**: "Bitcoin"
   - **Expected**: Values "Credit", "Debit", "Cash", "PayPal"
   - **Description**: Unacceptable payment method.
   - **Fix**: Change payment method to a valid option.

---

#### **Sheet: Products**
- **Issue #1**: Missing Data - **Major**
   - **Location**: Cell A3 (Row 3)
   - **Found**: Empty
   - **Expected**: Valid ProductCode
   - **Description**: Missing ProductCode for an entry.
   - **Fix**: Ensure ProductCode is provided for all products.

- **Issue #2**: Format Violation - **Critical**
   - **Location**: Cell D3 (Row 3), Column "StockQuantity"
   - **Found**: "-10"
   - **Expected**: Positive integer
   - **Description**: Negative stock quantity is invalid.
   - **Fix**: Adjust stock quantity to be a non-negative integer.

- **Issue #3**: Format Violation - **Major**
   - **Location**: Cell F3 (Row 3), Column "LastUpdated"
   - **Found**: "30/02/2024"
   - **Expected**: Valid date format (DD/MM/YYYY)
   - **Description**: Invalid date (February does not have 30 days).
   - **Fix**: Correct date entry to a valid format.

- **Issue #4**: Format Violation - **Minor**
   - **Location**: Cell D4 (Row 4), Column "Price"
   - **Found**: "-299.99"
   - **Expected**: Positive decimal value
   - **Description**: Price must be positive.
   - **Fix**: Adjust price to be a positive value.

- **Issue #5**: Format Violation - **Minor**
   - **Location**: Cell D3 (Row 3), Column "StockQuantity"
   - **Found**: "many"
   - **Expected**: Positive integer
   - **Description**: Non-numeric quantity detected.
   - **Fix**: Replace with a valid integer representing stock quantity.

---

### ACTIONABLE RECOMMENDATIONS
- Audit customer entries to ensure all IDs, names, and statuses adhere to the format and uniqueness requirements.
- Regularly validate order entries against defined rules including date and amount constraints.
- Update product entries to ensure all required columns have permissible values and formats.
- Use Excel functions to search for duplicates (e.g., `=COUNTIF()`), correct formats (e.g., `=TEXT()` for dates), and highlight or filter invalid entries as needed.

In [9]:
# Run the test
with trace("Data QC Agent Test 3"):
    await test_data_qc()

### EXECUTIVE SUMMARY

- **Total Issues by Severity**:
  - **Critical**: 8 issues
  - **Major**: 6 issues
  - **Minor**: 7 issues

- **Key Problem Areas**:
  - **Customer Data Sheet**: Missing values, invalid date formats, structural problems with IDs.
  - **Orders Sheet**: Inconsistent date formats, invalid total amounts, missing IDs.
  - **Products Sheet**: Issues with formats, including improper stock quantities and pricing.

---

### DETAILED FINDINGS

#### Sheet: Customer Data

**Issue #1**: Format violation - Critical
- **Location**: Cell B2 (First Row)
- **Found**: "2024/13/45"
- **Expected**: Valid date (DD/MM/YYYY)
- **Description**: Invalid date format found.
- **Fix**: Update the date format to conform to DD/MM/YYYY.

**Issue #2**: Missing data - Critical
- **Location**: Cell A3 (CustomerID)
- **Found**: `NaN`
- **Expected**: Required CustomerID value.
- **Description**: Missing CustomerID.
- **Fix**: Provide a valid CustomerID as per the required format.

**Issue #3**: Structural problem - Major
- **Location**: Cell A5 (CustomerID)
- **Found**: "CUST12348" 
- **Expected**: Format "CUST-XXXXX"
- **Description**: Invalid format for CustomerID.
- **Fix**: Reformat CustomerID to match the correct pattern.

**Issue #4**: Data consistency - Major
- **Location**: Cell B6 (FirstName)
- **Found**: `NaN`
- **Expected**: Required FirstName value.
- **Description**: Missing FirstName.
- **Fix**: Provide a valid FirstName.

**Issue #5**: Format violation - Minor
- **Location**: Cell E2 (Phone)
- **Found**: "555-234-5678"
- **Expected**: Format "(XXX) XXX-XXXX"
- **Description**: Phone number format does not match expected format.
- **Fix**: Update phone number to match the expected format.

---

#### Sheet: Orders

**Issue #1**: Missing data - Critical
- **Location**: Cell A6 (OrderID)
- **Found**: `NaN`
- **Expected**: Required OrderID value.
- **Description**: Missing OrderID.
- **Fix**: Provide a valid OrderID.

**Issue #2**: Format violation - Major
- **Location**: Cell C3 (OrderDate)
- **Found**: "32/13/2023"
- **Expected**: Valid date format (DD/MM/YYYY)
- **Description**: Invalid date format.
- **Fix**: Correct OrderDate to a valid format.

**Issue #3**: Data consistency - Major
- **Location**: Cell D2 (TotalAmount)
- **Found**: "450000"
- **Expected**: Less than 99999.99
- **Description**: TotalAmount exceeds the maximum limit.
- **Fix**: Adjust TotalAmount to be within the allowed range.

**Issue #4**: Format violation - Minor
- **Location**: Cell E2 (PaymentMethod)
- **Found**: "Bitcoin"
- **Expected**: Values "Credit", "Debit", "Cash", "PayPal"
- **Description**: PaymentMethod value is invalid.
- **Fix**: Replace with a valid PaymentMethod.

---

#### Sheet: Products

**Issue #1**: Missing data - Critical
- **Location**: Cell A1 (ProductCode)
- **Found**: `NaN`
- **Expected**: Required ProductCode.
- **Description**: Missing ProductCode.
- **Fix**: Provide a valid ProductCode.

**Issue #2**: Format violation - Major
- **Location**: Cell C3 (StockQuantity)
- **Found**: "-10"
- **Expected**: Non-negative integer
- **Description**: StockQuantity is negative.
- **Fix**: Correct StockQuantity to be non-negative.

**Issue #3**: Format violation - Minor
- **Location**: Cell E1 (LastUpdated)
- **Found**: "30/02/2024"
- **Expected**: Valid date format (DD/MM/YYYY)
- **Description**: Invalid date (February does not have 30 days).
- **Fix**: Update LastUpdated date to a legitimate date.

---

These findings outline critical issues that need immediate attention in order to ensure data integrity and compliance with the established specifications. Each fix can be easily implemented directly within Excel using the respective cell adjustments.